In [1]:
import pandas as pd
import numpy as np

In [2]:
def draw_sudoku(board):
    print("+" + "---+"*9)
    for i, row in enumerate(board):
        print(("|" + " {}   {}   {} |"*3).format(*[x if x != 0 else "0" for x in row]))
        if i % 3 == 2:
            print("+" + "---+"*9)

In [3]:
def read_csv(name):
    df = pd.read_csv(name, header=None)
    matrix = df.to_numpy()
    d=[]
    x = 0
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if(matrix[i,j] > 0):
                d.append( [[i,j],matrix[i,j]])
                x+=1
    print(fitness(matrix))
    draw_sudoku(matrix)
    return matrix, d

In [4]:
def initial_population(arr, n):    # generate n random matrix with values from 1->9
    x=y=val=-1
    population = []
    for i in range(n):
        population.append(np.zeros((9,9)))
    for j in range(n):
        mat = np.random.randint(low=1,high=10, size=(9, 9))
        #print(mat)
        for i in range(len(arr)):
            x = arr[i][0][0]
            y = arr[i][0][1]
            val = arr[i][1]
            mat[x,y] = val
        population[j] = mat
    return population

In [5]:
def fitness(m):
    side = m.shape[0]
    def repeated(st):
        st, c = np.unique(st, return_counts = True)
        if st[0] != 0: return np.count_nonzero(c > 1)
        return np.count_nonzero(c[1:] > 1)
    result = side**2
    for i in range(side):
        result -= repeated(m[i, :])
        result -= repeated(m[:, i])
        sqrt = np.sqrt(side)
        h, v = (i % sqrt)*sqrt, (i // sqrt)*sqrt
        result -= repeated(m[int(v): int(v + sqrt), int(h): int(h + sqrt)])
    return result - np.count_nonzero(m == 0)

In [14]:
def tournament(fitnesses, k = 2, reversed = False):
    participants = np.random.choice(len(fitnesses), k, False)
    if reversed: return np.argmin(fitnesses[participants].copy())
    return np.argmax(fitnesses[participants].copy())

In [7]:
def simplePermutationCrossover(pR, parents, d):
    if np.random.uniform(0, 1) < pR:
        return crossover(parents, d)
    return parents[0]

In [8]:
def crossover(parents, d):
    first, second = parents
    x = np.random.randint(low=0,high=9)
    offspring = np.zeros((9,9))
    offspring = offspring.astype(int)

    x=4 
    # merge matrixes
    for i in range(x+1):
        for j in range(offspring.shape[0]):
                offspring[i][j] = first[i][j]
    for i in range(x+1, offspring.shape[0]):
        for j in range(offspring.shape[1]):
                offspring[i][j] = second[i][j]
    #print(offspring)

    # missing-duplicated
    blocked = []
    for i in range(offspring.shape[0]):
        coords = []
        dupl = {}
        miss = [1,2,3,4,5,6,7,8,9]
        
        for j in range(offspring.shape[1]):
            coords.append([offspring[j,i], [i,j]])
        # print(coords)
        for j in range(len(coords)):
            rep = np.where(np.array(coords == coords[j][0], dtype=object))
            if(rep[0].shape[0] != 1):
                dupl[coords[j][0]] = rep[0]
        for j in range(len(coords)):
            if(coords[j][0] in miss):
                miss.remove(coords[j][0])
        for j in d:
            blocked.append(j[0])
        for j in dupl.values():
            for z in j:
                if([z,i] not in blocked): # si no está en la lista de bloqueados
                    # print([z,i], miss )
                    if(len(miss)):
                        offspring[z,i] = miss[0]
                        miss.pop(0)
    return offspring 

In [9]:
def mutation(pM, m):
    if np.random.uniform(0, 1) < pM:
        sqrt, side = int(np.sqrt(m.shape[0])), m.shape[0]
        for i in range(side):
            _, indices = np.unique(m[i], return_index=True)
            for j in range(side):
                if j not in indices and m[i, j]:
                    m[i, j], m[(i + sqrt) % side, j] = m[(i + sqrt) % side, j], m[i, j]
    return m

In [15]:
def geneticAlgorithm(f, n, g, pR, pM): #File name, Population Size, # of generations, P(reproduction), p(Mutation), # of queens
    m, d = read_csv(f)
    sideSize = m.shape[0]
    population = initial_population(d, n)
    best, fitnesses = -1, np.array([fitness(i) for i in population])
    best = np.argmax(fitnesses)
    newPopulation, newFitnesses = np.zeros((n, sideSize, sideSize)), np.zeros(n)
    for i in range(g):
        if i % 50 == 0: 
            r = population[best].astype(int)
            print(fitness(r))
            draw_sudoku(r)
        if fitnesses[best] == sideSize**2:
            return population[best].astype(int)
        for childIndex in range(n):
            pOne = population[tournament(fitnesses, n // 2)]
            pTwo = population[tournament(fitnesses, n // 2)]
            #pOne = population[roulette(population)]
            #pTwo = population[roulette(population)]
            parents = np.array([pOne, pTwo])
            newChild = simplePermutationCrossover(pR, parents, d)
            newChild = mutation(pM, newChild)
            newFitnesses[childIndex] = fitness(newChild)
            newPopulation[childIndex] = newChild
        betterBest = np.argmax(newFitnesses)
        #print(fitnesses[best] ,newFitnesses[betterBest])
        #print(fitnesses, newFitnesses)
        print("Before:", best, "->", fitnesses[best], betterBest, "->", newFitnesses[betterBest])
        if newFitnesses[betterBest] < fitnesses[best]: 
            print("Seeded")
            newPopulation[tournament(fitnesses, n//2, True)] = population[best]
        else:
            print("Updated")
            best = betterBest
        print(fitnesses, newFitnesses)
        population, fitnesses = newPopulation, newFitnesses
        print("After:", best, "->", fitnesses[best], betterBest, "->", newFitnesses[betterBest])
        print(fitnesses, newFitnesses)
    return population[best].astype(int)
r = geneticAlgorithm('easy2.csv', 300, 10, 0.75, 0.75)
print(fitness(r))
draw_sudoku(r)

38
+---+---+---+---+---+---+---+---+---+
| 0   9   8 | 2   0   0 | 4   0   6 |
| 0   1   5 | 3   9   4 | 8   0   0 |
| 4   2   0 | 0   0   0 | 5   1   0 |
+---+---+---+---+---+---+---+---+---+
| 0   0   0 | 4   7   0 | 1   0   0 |
| 1   0   7 | 0   8   0 | 0   9   3 |
| 0   0   6 | 0   3   0 | 0   0   5 |
+---+---+---+---+---+---+---+---+---+
| 8   0   0 | 0   0   3 | 6   7   1 |
| 3   0   4 | 6   0   8 | 0   0   0 |
| 9   0   0 | 0   0   5 | 0   8   0 |
+---+---+---+---+---+---+---+---+---+
33
+---+---+---+---+---+---+---+---+---+
| 4   9   8 | 2   5   4 | 4   7   6 |
| 6   1   5 | 3   9   4 | 8   4   4 |
| 4   2   2 | 8   4   4 | 5   1   6 |
+---+---+---+---+---+---+---+---+---+
| 2   8   1 | 4   7   9 | 1   7   5 |
| 1   4   7 | 1   8   6 | 3   9   3 |
| 7   5   6 | 2   3   4 | 1   3   5 |
+---+---+---+---+---+---+---+---+---+
| 8   5   2 | 7   1   3 | 6   7   1 |
| 3   8   4 | 6   6   8 | 3   5   3 |
| 9   7   8 | 4   7   5 | 2   8   7 |
+---+---+---+---+---+---+---+---+---+


/tmp/ipykernel_1390/1985218759.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rep = np.where(np.array(coords == coords[j][0], dtype=object))


Before: 31 -> 33 40 -> 51.0
Updated
After: 40 -> 51.0 40 -> 51.0
Before: 40 -> 39.0 28 -> 52.0
Updated
After: 28 -> 52.0 28 -> 52.0
Before: 28 -> 37.0 61 -> 49.0
Updated
After: 61 -> 49.0 61 -> 49.0
Before: 61 -> 37.0 6 -> 51.0
Updated
After: 6 -> 51.0 6 -> 51.0
Before: 6 -> 33.0 221 -> 50.0
Updated
After: 221 -> 50.0 221 -> 50.0
Before: 221 -> 38.0 181 -> 50.0
Updated
After: 181 -> 50.0 181 -> 50.0
Before: 181 -> 41.0 59 -> 48.0
Updated
After: 59 -> 48.0 59 -> 48.0
Before: 59 -> 39.0 66 -> 48.0
Updated
After: 66 -> 48.0 66 -> 48.0
Before: 66 -> 36.0 102 -> 54.0
Updated
After: 102 -> 54.0 102 -> 54.0
Before: 102 -> 41.0 81 -> 51.0
Updated
After: 81 -> 51.0 81 -> 51.0
51
+---+---+---+---+---+---+---+---+---+
| 6   8   9 | 8   2   5 | 1   4   8 |
| 2   1   7 | 6   3   4 | 7   5   1 |
| 4   2   1 | 9   4   9 | 6   9   2 |
+---+---+---+---+---+---+---+---+---+
| 7   3   8 | 1   1   2 | 4   6   9 |
| 1   9   5 | 3   7   8 | 5   2   6 |
| 5   6   4 | 2   9   7 | 8   7   3 |
+---+---+---+---+